# 동적 웹크롤링

### 라이브러리 설치

In [ ]:
!pip install selenium
!pip install bs4
!pip install tqdm

## WebDriver 임포트

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 객체생성
wd = webdriver.Chrome()   # 자동화 브라우저 생성

In [ ]:
# 자동화 브라우져에 주소를 입력
wd.get("https://www.coffeebeankorea.com/store/store.asp")

In [ ]:
# 해당 주소에 있는 자바스크립트 실행
wd.execute_script("storePop2(0)")

In [ ]:
# html 소스로 변환
html = wd.page_source

In [ ]:
# 파싱을 위해 beautifulsoup 객체 생성
soupCB1 = BeautifulSoup(html,'html.parser')

In [ ]:
# html 확인 - 필요하면
# print(soupCB1.prettify())

In [ ]:
# 추출하려는 패턴을 확인하고 적용  매장명
store_title = soupCB1.select('div.store_txt>h2')
len(store_title) > 0

In [ ]:
# 매장정보
tr_lists = soupCB1.select('table.store_table>tbody>tr')

In [ ]:
print(tr_lists[0].th.text,  tr_lists[0].td.text)
print(tr_lists[1].th.text,  tr_lists[1].td.text)
print(tr_lists[2].th.text,  tr_lists[2].td.text)
print(tr_lists[3].th.text,  tr_lists[3].td.text)
print(tr_lists[4].th.text,  tr_lists[4].td.text)

# 한꺼번에 실행하기

In [173]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm # 진행상태를 표시
import pandas as pd

In [176]:
# 동적페이지 크롤링
wd = webdriver.Chrome()  # 자동화 크롬브라우져 생성
store_lists = []
for store_num in tqdm(range(100)):
    try:
        wd.get("https://www.coffeebeankorea.com/store/store.asp")  # 자동화 브라우져에 주소를 입력
        time.sleep(1)  # 대기
        
        wd.execute_script(f"storePop2({store_num})")  # 해당 화면에 내장되어있는 javasscript:storePop2('3') 를 코드로 실행  팝업창이 뜸
        time.sleep(1)   # 팝업창이 완전히 로드될때까지 1초간 대기
        
        html = wd.page_source  # 브라우져의 내장 html 코드를 반환
        soupCB1 = BeautifulSoup(html,'html.parser')  # 파싱할수 있도록 객체를 생성
        store_title = soupCB1.select('div.store_txt>h2')[0].text #찾고자하는 경로에 해당하는 데이터를 추출
        if store_title == '':  # 매장정보가 없으면
            continue  #  아래 문장을 실행하지 안히고 for문 처음으로 순환
        # 매장정보
        tr_lists = soupCB1.select('table.store_table>tbody>tr')      # 매장정보를 가지는 tr 리스트를 추출
        store_lists.append([store_title, tr_lists[0].td.text, tr_lists[1].td.text, tr_lists[2].td.text, tr_lists[3].td.text,tr_lists[4].td.text])       
        
        # # 버튼요소를 찾기  닫으려는 대상 객체가 없으면 예외가발생
        # button = WebDriverWait(wd,1).until(
        #     EC.presence_of_all_elements_located( (By.ID,'closeBtn1ActiveBtn'))
        # )[0]
        # button.click()
    except Exception as e:
        pass
        
# 매장정보 추출        
df = pd.DataFrame(store_lists,columns=['매장명','영업시간','주차','주소','전화번호','기타정보'])
df.to_csv('coffebean_store_lists.csv', encoding='cp949')
df

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [04:57<00:00,  2.97s/it]


,매장명,영업시간,주차,주소,전화번호,기타정보
0,차병원점,평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무,병원주차장 이용가능 (병원방문시 무료),서울시 강남구 논현로 566 강남차병원1층,02-538-7615,식약처 인증 위생등급 매우 우수매장
1,청담성당점,월-금 07:00~21:00 | 토 07:30~21:00 | 일/공휴일 07:30~...,발렛주차가능 / 최초 1시간 4000원 / 1시간당 추가시 2000원,서울시 강남구 삼성로 716 LEE76빌딩2층,02-542-2053,홀케익 당일 수령가능
2,서초우성점,평일 07:00~23:00ㅣ토 07:00~22:30 I 일/공휴일 08:00~22:00,,서울시 서초구 강남대로 51길 1 511 TOWER 1층,02-582-0158,홀케익 당일 수령가능
3,삼성봉은사거리점,평일 07:00-22:30 | 주말/공휴일 07:00-22:00,"건물뒷편주차장(기계주차식)평일 최초 30분 2,000원 / 1시간 5,000원 (구...","서울시 강남구 영동대로 607 1,2층",02-3443-5618,식약처 인증 위생등급 매우 우수매장
4,잠실신천점,월-금 07:00~22:00 | 주말&공휴일 07:00~21:00,1시간 무료주차가능-건물옆에 주차타워(기계주차식)-대형차량불가반려동물 출입 가능,서울시 송파구 석촌호수로 118 1층,02-416-9520,펫 프렌들리 매장
5,삼성전자강남사옥점,월-금 07:00~17:00 | 토/일/공휴일 휴점,1시간 무료주차 (영수증 지참시),서울시 서초구 서초대로74길 11 지하2층,02-2055-1142,식약처 인증 위생등급 매우 우수매장
6,삼성생명강남사옥점,월-금 07:00~18:00 | 토/일/공휴일 휴점,주차정보1시간 무료주차,서울시 서초구 서초대로74길 4 삼성생명보험서초타워내지하1층,02-581-5562,식약처 인증 위생등급 매우 우수매장
7,방이점,매일 7:00 - 24:00,주차불가,서울시 송파구 오금로 11길 7,02-415-5561,홀케익 당일 수령가능
8,광화문점,월-금 07:00~23:00 | 토요일 7:30~23:00 | 일/공휴일 09:00...,"브루잉 바 이용 가능유료주차 최초 30분 2,000원, 10분당 1,000원",서울 종로구 서린동 136 센트리얼빌딩 1층,02-3210-2326,홀케익 당일 수령가능
9,경희궁의아침점,월-금 07:00-21:00 | 토요일 09:00-21:00 | 일요일/공휴일 09...,건물지하3층 주차장( 1시간 무료주차 가능),서울시 종로구 새문안로3길 23 경희궁아침상가4단지105외,02-730-8896,홀케익 당일 수령가능


In [175]:
store_lists

[['차병원점',
  ' 평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무',
  '병원주차장 이용가능 (병원방문시 무료)',
  '서울시 강남구 논현로 566 강남차병원1층  ',
  '02-538-7615',
  '식약처 인증 위생등급 매우 우수매장'],
 '차병원점']

# 예외처리

In [ ]:
for i in ['1','2','abc','3']:
    try:
        print(int(i))
    except Exception as e:
        print("예외가 발생했습니다.", e)

In [ ]:
for i in ['1','2','abc','3']:    
        print(int(i))
    